In [ ]:
import keras, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, tqdm
import os.path
import time
%matplotlib inline

In [ ]:
import nnUtils

In [ ]:
import sklearn, sklearn.model_selection

In [ ]:
datadir = os.path.join(os.path.expanduser('~'), 'data2', 'behavioralCloning')

In [ ]:
# data = np.load(os.path.join(datadir, 'multiData-mouseForwardFuller.npz'))
# X = data['X']
# Y = data['Y'][:, 0].reshape(-1, 1)

In [ ]:
!ls ~/data2/behavioralCloning/*.zip

In [ ]:
import loadData, models

from importlib import reload
reload(loadData);
reload(models);

In [ ]:
dataGenerator = loadData.DataGenerator(
    [
        os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
        for p in (
            'mouseForward.zip',
            'mouseReverse.zip',
            'longCarefulForward.zip',
            'longCarefulReverse.zip',
#             'dirtSidesForward.zip',
#             'dirtSidesReverse.zip',
#             'data_provided.zip',
        )
    ],
    responseKeys=('steering',),# 'throttle', 'brake',)
    #verbose=False,
)

In [ ]:
NEPOCH = 42

In [ ]:
modelname = 'cropped_nvidia_unactivatedConv_reluFC_finaltanh-kl2p0_bl2p0-mouseLongCare-%depoch' % NEPOCH

In [ ]:
%%time
#kernel_regularizer = keras.regularizers.l2(0.0001)
#bias_regularizer = keras.regularizers.l2(0.01)
Model = models.Nvidia

model = Model(
    1, dataGenerator.sampleImageShape,
    optimizer=keras.optimizers.Nadam(
        lr=0.0001, #beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    ),
    #kernel_regularizer=kernel_regularizer,
    #bias_regularizer=bias_regularizer,
)

In [ ]:
trainGen = dataGenerator.generate(epochSubsampling=1.0)
validGen = dataGenerator.generate(validation=True)

In [ ]:
# def trainYielder():
#     while True:
#         yield trainGen.__next__()

# for x in trainYielder():
#     print([y.shape for y in x])

In [ ]:
%%time
history = nnUtils.fitModelWithDataGenerator(model, dataGenerator, modelname, epochs=NEPOCH)

Retrain on some other data.

In [ ]:
# %%time
# dataGenerator = loadData.DataGenerator(
#     [
#         os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
#         for p in (
#             'longCarefulForward.zip',
#             'longCarefulReverse.zip',
#         )
#     ],
#     responseKeys=('steering',),
# )
# NEPOCH = 4
# modelname = 'cropped_nvidia_unactivatedConv_reluFC_finaltanh-kl2p0_bl2p0-longCareRetrain-%depoch' % NEPOCH
# trainGen = dataGenerator.generate(epochSubsampling=1.0)
# validGen = dataGenerator.generate(validation=True)
# history = nnUtils.fitModelWithDataGenerator(model, dataGenerator, modelname, epochs=NEPOCH)

In [ ]:
%%time
fpath = os.path.join(datadir, '%s.h5' % modelname)
print('Saving to', fpath)
model.save(fpath)

In [ ]:
!recent /home/tsbertalan/data2/behavioralCloning/ | head -n 4

In [ ]:
modelname

In [ ]:
trainIndices, validationIndices = dataGenerator._indices
for ind, lab in zip(dataGenerator._indices, ('train', 'validation')):
    
    imgs = []
    responses = []
    for k in range(ind[0], ind[0]+1000):
        try:
            image, response = dataGenerator.getImageResponse(k)
            imgs.append(image.reshape((1, *image.shape)))
            responses.append(response)
        except KeyError:
            break
    X = np.vstack(imgs)
    Y = np.vstack(responses)

    pred = model.predict(X)
    pred.shape, Y.shape

    from collections import deque
    def runningMean(x, N):
        y = []
        hist = deque(maxlen=N)
        for a in x:
            hist.append(a)
            y.append(np.mean(hist))
        return np.array(y)
    
    ind = np.copy(ind)
    ind.sort()
    fig, ax = plt.subplots()
    start = 0
    end = start + 200
    ax.plot(Y[start:end], label=r'truth $\theta$', color='black')
    ax.plot(pred[start:end], label=r'predictions $\hat\theta$', color='red')
    filtersize = 3
    scale = 1
    smoothed = scale * runningMean(pred, filtersize)
    ax.plot(
        smoothed[start:end], 
        label=r'$\hat\rho = %.1f \cdot box_{%d}(\hat\theta)$' % (scale, filtersize),
        alpha=.25,
        color='red',
    )
    ax.set_xlabel('time [samples]')
    ax.set_ylabel('angle [radians]')
    ax.set_title(lab)
    ax.legend();
    fig.savefig('smoothingEffect-%s-%s.png' % (modelname, lab))